<a href="https://colab.research.google.com/github/RithikaBaskaran/Research-Assistant-using-Machine-Learning/blob/main/Pizza_Hub_Management_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import os
from datetime import datetime
from tabulate import tabulate
#to install tabulate , in command prompt , goto python dir, type   py -m pip install tabulate

# File paths
MENU_FILE = 'menu.csv'
TOPPINGS_FILE = 'toppings.csv'
SIDE_ORDERS_FILE = 'side_orders.csv'
OFFERS_FILE = 'offers.csv'
ORDERS_FILE = 'orders.csv'
ORDER_ITEMS_FILE = 'order_items.csv'
DELIVERED_ORDERS_FILE = 'delivered_orders.csv'
USERS_FILE = 'users.csv'
CART_FILE = 'cart.csv'

def read_csv(file_path):
    if not os.path.exists(file_path):
        return []
    with open(file_path, 'r', newline='') as file:
        reader = csv.DictReader(file)
        return list(reader)

def write_csv(file_path, data, fieldnames):
    with open(file_path, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

def get_next_id(file_path):
    data = read_csv(file_path)
    if not data: return 1
    return max(int(item['id']) for item in data) + 1

def register_user():
    users = read_csv(USERS_FILE)
    user_id = get_next_id(USERS_FILE)
    name=input("Enter your name: ")
    username = input("Enter username: ")
    password = input("Enter password: ")
    ph_no= input("Enter your phone number: ")
    address=input("Enter your address: ")
    email=input("Enter Email id: ")
    if any(not value for value in [name, username,password,ph_no,address,email]):
        print("Please try again")
        return
    if any(user['username'] == username for user in users):
        print("Username already exists.")
        return

    users.append({
        'id': user_id,
        'name': name,
        'username': username,
        'password': password,
        'phone': ph_no,
        'address': address,
        'email': email
    })
    write_csv(USERS_FILE, users, fieldnames=['id', 'name','username', 'password', 'phone','address', 'email'])
    print("User registered successfully!")

def login():
    users = read_csv(USERS_FILE)
    username = input("Enter username: ")
    password = input("Enter password: ")
    f=0
    for user in users:
        if user['username'] != username:
            continue
        else:
            f=1
            u=user['username']
            p=user['password']
            break
    if f==1:
        if p == password:
            print(f"Welcome {user['name']}!")
            return user['id']
        else:
            f=input("Forgot Password (yes/no): ")
            if f=='yes':
                new_password = input("Enter your new password: ")
                user['password'] = new_password
                write_csv(USERS_FILE, users, fieldnames=['id', 'name','username', 'password', 'phone', 'address','email'])
                return
            else:
                print("Pls try again")
                return
    else:
        f=input("Forgot Username (yes/no): ")
        if f=='yes':
            email = input("Enter your registered email address: ")
            if user['email'] == email:
                print(f"Your username is: {user['username']}")
                return
            print("Email address not found. Please try again.")
        else:
            print("Pls try again")
            return

def user_menu(user_id):
    if user_id:
        print("\nWelcome to Pizza Hub!!!")
        while True:
            print("1. View Pizza Hub Pamplet")
            print("2. Add to Cart  3. View Cart  4. Clear Cart  5. Place Order")
            print("6. View Ordered Details  7. Update Profile 8. Exit")
            customer_choice = input("Enter your choice: ")
            if customer_choice == '1':
                display_items(MENU_FILE, "Pizza Varieties")
                display_items(TOPPINGS_FILE, "Toppings")
                display_items(SIDE_ORDERS_FILE, "Snacks and Refreshments")
            elif customer_choice == '2': add_to_cart(user_id)
            elif customer_choice == '3': view_cart(user_id)
            elif customer_choice == '4': clear_cart(user_id)
            elif customer_choice == '5': place_order(user_id)
            elif customer_choice == '6': view_order_history(user_id)
            elif customer_choice == '7': update_profile(user_id)
            elif customer_choice == '8': break
            else: print("Invalid choice. Try again.")

# Function to update user profile information
def update_profile(user_id):
    users = read_csv(USERS_FILE)
    for user in users:
        if user['id'] == user_id:
            print("Your Profile:")
            print(f"Name: {user['name']}, Username: {user['username']}, Phone Number: {user['phone']}, Address: {user['address']}")
            print("\nUpdate Profile:")
            print("1. Phone Number  2. Address  3. Change Password  4. Email id  5. None")
            choice = input("Enter your choice: ")
            if choice == '1':
                phone_number = input("Enter new phone number: ")
                user['phone'] = phone_number
                print("Phone number updated successfully!")
            elif choice == '2':
                address = input("Enter new address: ")
                user['address'] = address
                print("Address updated successfully!")
            elif choice == '3':
                current_password = input("Enter current password: ")
                if current_password == user['password']:
                    new_password = input("Enter new password: ")
                    confirm_password = input("Confirm new password: ")
                    if new_password == confirm_password:
                        user['password'] = new_password
                        print("Password changed successfully!")
                    else: print("Passwords do not match.")
                else: print("Incorrect current password.")
            elif choice == '4':
                new_email = input("Enter new email id: ")
                user['email'] = new_email
                print("Email changed successfully!")
            elif choice==5: break
            else: print("Invalid choice.")
            break
    write_csv(USERS_FILE, users, fieldnames=['id', 'name','username', 'password', 'phone', 'address','email'])

def add_to_cart(user_id):
    menu = read_csv(MENU_FILE)
    toppings = read_csv(TOPPINGS_FILE)
    side_orders = read_csv(SIDE_ORDERS_FILE)
    cart = read_csv(CART_FILE)
    order_items = []
    total_price = 0
    while True:
        display_items(MENU_FILE, "Menu Items")
        item_id = input("Enter the menu item ID to order or type 'done' to finish: ")
        if item_id.lower() == 'done': break
        item = next((i for i in menu if i['id'] == item_id), None)
        quantity=0
        if not item:
            print("please give right input.")
            continue
        try:
            quantity = int(input(f"Enter quantity for {item['name']} pizza: "))
        except ValueError:
            print("quantity not entered. pls try again")
            continue
        if quantity==0:
            print("quantity not entered. pls try again")
            continue
        item_total_price = int(quantity) * float(item['price'])
        selected_toppings = []
        toppings_total_price = 0.0
        if item['category'].lower() == 'pizza':
            add_top = input("Do you want to add toppings? (yes/no): ").lower()
            if add_top not in ['yes','no']:
                print("please give right input.")
                continue
            if add_top == 'yes':
                while True:
                    print("\nAvailable Toppings:")
                    print(tabulate(toppings, headers="keys", tablefmt="grid"))
                    topping_id = input("Enter the topping ID to add or type 'done' to finish: ")
                    if topping_id.lower() == 'done' or topping_id == '': break
                    topping = next((t for t in toppings if t['id'] == topping_id), None)
                    if not topping:
                        print("Topping not found.")
                        continue
                    #topping_quantity = int(input(f"Enter quantity for {topping['name']}: "))
                    selected_toppings.append(topping['name'])
                    toppings_total_price += float(topping['price'])
            if not selected_toppings:
                selected_toppings.append('None')
            item_total_price += toppings_total_price * quantity
            total_price += item_total_price

            order_items.append({
            'menu_id': item_id,
            'category':'pizza',
            'quantity': quantity,
            'toppings': selected_toppings,
            'price': item_total_price
        })
    if not order_items:
        print("No Pizza selected.")

    side_orders = []
    while True:
        display_items(SIDE_ORDERS_FILE, "Side Orders")
        side_order_id = input("Enter the side order ID to order or type 'done' to finish: ")
        if side_order_id.lower() == 'done':
            break
        side_order = next((s for s in read_csv(SIDE_ORDERS_FILE) if s['id'] == side_order_id), None)
        if not side_order:
            print("Side order not found.")
            continue
        try:
            qty = int(input(f"Enter quantity for {side_order['name']}: "))
        except ValueError:
            print("quantity not entered. pls try again")
            continue
        if quantity==0:
            print("quantity not entered. pls try again")
            continue
        side_orders.append({
            'side_order_name': side_order['name'],
            'quantity': quantity
        })

        side_total_price =float(side_order['price'])* qty
        total_price +=side_total_price

        order_items.append({
            'menu_id': side_order_id,
            'category':'side_orders',
            'quantity': qty,
            'toppings': ['None'],
            'price': side_total_price
        })

    if not order_items:
        print("No items selected.")
        return

    if total_price > 0.0:
        for item in order_items:
            cart.append({
                'user_id': user_id,
                'menu_id': item['menu_id'],
                'category': item['category'],
                'quantity': item['quantity'],
                'toppings': ', '.join(item['toppings']),
                'price': item['price']
            })
        write_csv(CART_FILE, cart, fieldnames=['user_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
        print(f"Items added to cart successfully! Total price: ₹{total_price}")
    else: print("No items selected. Cart is not updated.")

def place_order(user_id):
    cart = read_csv(CART_FILE)
    offers = read_csv(OFFERS_FILE)
    user_cart = [item for item in cart if item['user_id'] == user_id]
    if not user_cart:
        print("Your cart is empty. Please add items to cart before placing an order.")
        return
    total_price = sum(float(item['price']) for item in user_cart if item['category'] == 'pizza' )

    display_items(OFFERS_FILE, "Offers")
    offer_id = input("Enter offer ID if you have one or type 'none': ")
    offer = next((o for o in offers if o['id'] == offer_id), None)
    discount=0.0
    if offer:
        offer_end_date = datetime.strptime(offer['end_date'], '%Y-%m-%d')
        if datetime.now() > offer_end_date:
            print("Offer expired!")
            return
        discount = float(offer['discount']) / 100
        total_price -= total_price * discount

        print("Price of pizza after discount: ",total_price)

    tot_price= sum(float(item['price']) for item in user_cart if item['category'] == 'side_orders' )
    total_price +=tot_price

    print("Total Price: ", total_price)

    order_id = get_next_id(ORDERS_FILE)
    orders = read_csv(ORDERS_FILE)
    orders.append({
        'id': order_id,
        'user_id': user_id,
        'total_price': total_price,
        'order_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'paid': 'yes',
    })

    write_csv(ORDERS_FILE, orders, fieldnames=['id', 'user_id', 'total_price', 'order_time', 'paid'])

    order_items_data = read_csv(ORDER_ITEMS_FILE)
    for item in user_cart:
        order_items_data.append({
            'order_id': order_id,
            'menu_id': item['menu_id'],
            'category':item['category'],
            'quantity': item['quantity'],
            'toppings': item['toppings'],
            'price': item['price']
        })
    write_csv(ORDER_ITEMS_FILE, order_items_data, fieldnames=['order_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
    generate_and_send_receipt(user_id, user_cart, total_price)

    cart = [item for item in cart if item['user_id'] != user_id]  # Clear user's cart
    write_csv(CART_FILE, cart, fieldnames=['user_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
    print("Order placed successfully!")

def generate_and_send_receipt(user_id, user_cart, total_price):
    users = read_csv(USERS_FILE)
    user = next((u for u in users if u['id'] == user_id), None)

    if user:
        receipt = f"Bill Receipt\n\nCustomer: {user['name']}\nPhone: {user['phone']}\n"
        receipt += f"Order Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        receipt += "\nItems:\n"

        menu = read_csv(MENU_FILE)
        side_orders = read_csv(SIDE_ORDERS_FILE)

        for item in user_cart:
            item_id = item['menu_id']
            category = item['category']
            if category == 'pizza':
                menu_item = next((i for i in menu if i['id'] == item_id), None)
                item_name = menu_item['name'] if menu_item else 'Unknown Pizza'
            else:
                side_item = next((i for i in side_orders if i['id'] == item_id), None)
                item_name = side_item['name'] if side_item else 'Unknown Side Order'

            receipt += f"{item_name} x {item['quantity']} - ₹{item['price']}\n"
            if item['toppings']:
                receipt += f"Toppings: {item['toppings']}\n"

        receipt += f"\nTotal Price: ₹{total_price}\n"

        print("\nSending Bill Receipt to Customer...\n")
        print(receipt)

# Function to view the order history
def view_order_history(user_id):
    orders = read_csv(ORDERS_FILE)
    order_items = read_csv(ORDER_ITEMS_FILE)
    menu = read_csv(MENU_FILE)
    side_orders = read_csv(SIDE_ORDERS_FILE)

    user_orders = [order for order in orders if order['user_id'] == user_id]
    if not user_orders:
        print("No order history found.")
        return
    print("\nYour Order History:")
    for order in user_orders:
        order_id = order['id']
        print(f"Order ID: {order_id}, Total Price: ₹{order['total_price']}, Order Time: {order['order_time']}")
        print("Items:")
        for item in order_items:
            if item['order_id'] == order_id:
                item_id = item['menu_id']
                category=item['category']
                m_id = next((i for i in menu if i['id'] == item_id and i['category']==category), None)
                if m_id !=None:
                    name=m_id['name']
                else:
                    m_id = next((i for i in side_orders if i['id'] == item_id), None)
                    name=m_id['name']
                print(f"Item Name: {name}, Quantity: {item['quantity']}, Toppings: {item['toppings']}, Price: ₹{item['price']}")

# Function to view the cart
def view_cart(user_id):
    cart = read_csv(CART_FILE)
    menu = read_csv(MENU_FILE)
    side_orders = read_csv(SIDE_ORDERS_FILE)
    user_cart = [item for item in cart if item['user_id'] == user_id]

    if not user_cart:
        print("Your cart is empty.")
        return
    total_price = sum(float(item['price']) for item in user_cart)

    print("\nYour Cart:")
    for item in user_cart:
        item_id = item['menu_id']
        category=item['category']
        m_id = next((i for i in menu if i['id'] == item_id and i['category']==category), None)
        if m_id !=None:
            name=m_id['name']
        else:
            m_id = next((i for i in side_orders if i['id'] == item_id), None)
            name=m_id['name']
        if item['category']=='pizza' and item['toppings'] !='None':
            print(f"Item Name: {name}, Quantity: {item['quantity']}, Toppings Added: {item['toppings']}, Price: ₹{item['price']}")
        else:
            print(f"Item Name: {name}, Quantity: {item['quantity']},  Price: ₹{item['price']}")
    print(f"Total Price: ₹{total_price}")

def clear_cart(user_id):
    cart = read_csv(CART_FILE)
    updated_cart = [item for item in cart if item['user_id'] != user_id]
    # Write the updated cart back to the file, removing the user's items
    write_csv(CART_FILE, updated_cart, fieldnames=['user_id', 'menu_id', 'quantity', 'price', 'toppings'])
    print("Cart cleared successfully!")

def admin_panel():
    passcode = input("Enter admin panel passcode: ")
    if passcode == 'a1b2c3':
        while True:
            print("\nAdmin Panel")
            print("1. View  2. Add  3.Remove  4.Update  5.Exit")
            choice = input("Enter choice: ")
            if choice == '1': view_options()
            elif choice == '2': add_options()
            elif choice == '3': remove_options()
            elif choice == '4': update_options()
            elif choice == '5': break
            else: print("Invalid choice. Try again.")
    else: print("Incorrect passcode. Access denied.")

def view_options():
    while True:
        print("\nView Options")
        print("1. Menu Items  2. Offers  3. Orders to Deliver  4. Delivered Orders  5. View Users  6. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1':
            display_items(MENU_FILE, "Menu Items")
            display_items(TOPPINGS_FILE, "Toppings")
            display_items(SIDE_ORDERS_FILE, "Side Orders")
        elif choice == '2': display_items(OFFERS_FILE, "Offers")
        elif choice == '3': display_items(ORDERS_FILE, "Orders to Deliver")
        elif choice == '4': display_items(DELIVERED_ORDERS_FILE, "Delivered Orders")
        elif choice == '5': display_user_details()
        elif choice  == '6': break
        else: print("Invalid choice. Try again.")

def add_options():
    while True:
        print("\nAdd Options")
        print("1. Menu Item  2. Topping  3. Side Order  4. Offer  5. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1': add_menu_item()
        elif choice == '2': add_topping()
        elif choice == '3': add_side_order()
        elif choice == '4': add_offer()
        elif choice == '5': break
        else: print("Invalid choice. Try again.")

def remove_options():
    while True:
        print("\nRemove Options")
        print("1. Menu Item  2. Topping  3. Side Order  4. Offer  5. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1': remove_item(MENU_FILE, "Menu Item")
        elif choice == '2': remove_item(TOPPINGS_FILE, "Topping")
        elif choice == '3': remove_item(SIDE_ORDERS_FILE, "Side Order")
        elif choice == '4': remove_item(OFFERS_FILE, "Offer")
        elif choice == '5': break
        else: print("Invalid choice. Try again.")

def update_options():
    while True:
        print("\nUpdate Options")
        print("1. Menu Item Price 2. Topping Price  3. Side Order Price  4. Offer Discount / End Date")
        print("5. Mark Orders as Delivered  6. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1': update_item(MENU_FILE, "Menu Item")
        elif choice == '2': update_item(TOPPINGS_FILE, "Topping")
        elif choice == '3': update_item(SIDE_ORDERS_FILE, "Side Order")
        elif choice == '4': update_offer()
        elif choice == '5': mark_order_as_delivered()
        elif choice == '6': break
        else: print("Invalid choice. Try again.")

def mark_order_as_delivered():
    display_items(ORDERS_FILE, "Orders to Deliver")
    while True:
        order_id = input("Enter the Order ID to mark as delivered or type 'done':: ")
        if order_id.lower() == 'done':
            break
        if order_id:
            orders = read_csv(ORDERS_FILE)
            order_items = read_csv(ORDER_ITEMS_FILE)
            delivered_orders = read_csv(DELIVERED_ORDERS_FILE)

            order = next((o for o in orders if o['id'] == order_id), None)
            if not order:
                print("Order ID not found.")
                continue
            d=get_next_id(DELIVERED_ORDERS_FILE)

            # Prepare the delivered order entry
            delivered_order = {
                'id': d,
                'order_id':order['id'],
                'user_id': order['user_id'],
                'total_amount': order['total_price'],
                'delivered_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            delivered_orders.append(delivered_order)
            write_csv(DELIVERED_ORDERS_FILE, delivered_orders, fieldnames=['id', 'order_id','user_id', 'total_amount', 'delivered_date'])

        # Remove the order from orders.csv
        orders = [o for o in orders if o['id'] != order_id]
        write_csv(ORDERS_FILE, orders, fieldnames=['id', 'user_id', 'total_price', 'order_time', 'paid'])
        # Remove the corresponding items from order_items.csv
        order_items = [item for item in order_items if item['order_id'] != order_id]
        write_csv(ORDER_ITEMS_FILE, order_items, fieldnames=['order_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
        print(f"Order {order_id} marked as delivered.")

def display_items(file_path, item_type):
    items = read_csv(file_path)
    if not items:
        print(f"No {item_type.lower()} available.")
        return
    print(f"\n{item_type}:")
    print(tabulate(items, headers="keys", tablefmt="grid"))

def display_user_details():
    users = read_csv(USERS_FILE)
    # Extract headers excluding the password
    headers = [key for key in users[0].keys() if key != 'password']
    # Extract data excluding the password
    user_data = [
        {key: user[key] for key in headers} for user in users
    ]
    # Use tabulate to display the user details in a table format
    if user_data:
        print(tabulate(user_data, headers="keys", tablefmt="grid"))
    else:
        print("No user details available.")



def add_menu_item():
    menu = read_csv(MENU_FILE)
    item_id = get_next_id(MENU_FILE)
    name = input("Enter the name of pizza: ")
    while True:
        price = input("Enter the price: ")
        if (not price.isdigit()) or float(price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break
    category = 'pizza'
    typ=input("Enter type of pizza(veg/non veg): ")
    size=input("Enter size(e.g. small/medium/large): ")
    if any(not value for value in [name, price, typ, size]):
        print("Please try again")
        return
    menu.append({
        'id': item_id,
        'name': name,
        'category': category,
        'type': typ,
        'size': size,
        'price': price

    })
    write_csv(MENU_FILE, menu, fieldnames=['id', 'name', 'category', 'type','size', 'price'])
    print("Menu item added successfully!")

def add_topping():
    toppings = read_csv(TOPPINGS_FILE)
    topping_id = get_next_id(TOPPINGS_FILE)
    name = input("Enter the topping name: ")
    while True:
        price = input("Enter the price: ")
        if not price.isdigit() or float(price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break
    if any(not value for value in [name, price]):
        print("Please try again")
        return

    toppings.append({
        'id': topping_id,
        'name': name,
        'price': price
    })
    write_csv(TOPPINGS_FILE, toppings, fieldnames=['id', 'name', 'price'])
    print("Topping added successfully!")

def add_side_order():
    side_orders = read_csv(SIDE_ORDERS_FILE)
    side_order_id = get_next_id(SIDE_ORDERS_FILE)
    name = input("Enter the side order name: ")
    while True:
        price = input("Enter the price: ")
        if not price.isdigit() or float(price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break
    size =input("Enter size(e.g. small/medium/4 count/6 pieces: ")
    category = input("Enter the category(drink/chicken wings/bread: ")
    if any(not value for value in [name, price, size, category]):
        print("Please try again")
        return

    side_orders.append({
        'id': side_order_id,
        'name': name,
        'category': category,
        'size': size,
        'price': price

    })
    write_csv(SIDE_ORDERS_FILE, side_orders, fieldnames=['id', 'name', 'category','size', 'price'])
    print("Side order added successfully!")

def add_offer():
    offers = read_csv(OFFERS_FILE)
    offer_id = get_next_id(OFFERS_FILE)
    offer_name= input("Enter offer name: ")
    while True:
        discount = input("Enter the discount percentage: ")
        if not discount.isdigit() or int(discount)==0:
            print("Please enter a valid number for the discount")
            continue
        else: break
    while True:
        try:
            end_date = input("Enter the end date (YYYY-MM-DD): ")
            val_date= datetime.strptime(end_date, "%Y-%m-%d")
            break
        except ValueError:
            print("Please enter a valid date for the discount")
            continue
    if any(not value for value in [offer_name, discount, end_date]):
        print("Please try again")
        return

    offers.append({
        'id': offer_id,
        'name': offer_name,
        'discount': discount,
        'end_date': end_date
    })
    write_csv(OFFERS_FILE, offers, fieldnames=['id','name', 'discount', 'end_date'])
    print("Offer added successfully!")

def remove_item(file_path, item_type):
    display_items(file_path, item_type)
    items = read_csv(file_path)
    item_id = input(f"Enter the {item_type.lower()} ID to remove: ")
    updated_items = [item for item in items if item['id'] != item_id]
    write_csv(file_path, updated_items, fieldnames=items[0].keys())
    print(f"{item_type} removed successfully!")

def update_item(file_path, item_type):
    display_items(file_path, item_type)
    items = read_csv(file_path)
    item_id = input(f"Enter the {item_type.lower()} ID to update price: ")
    item = next((i for i in items if i['id'] == item_id), None)
    if not item:
        print(f"{item_type} not found.")
        return
    while True:
        new_price = input(f"Enter new price for {item['name']}: ")
        if not new_price.isdigit() or float(new_price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break

    item['price'] = new_price
    write_csv(file_path, items, fieldnames=items[0].keys())
    print(f"{item_type} price updated successfully!")

def update_offer():
    offers = read_csv(OFFERS_FILE)
    offer_id = input("Enter the offer ID to update: ")
    offer = next((o for o in offers if o['id'] == offer_id), None)
    if not offer:
        print("Offer not found.")
        return
    print("1. Update Discount Percentage  2-New End date ")
    choice = input("Enter your choice: ")
    if choice==1:
        while True:
            new_discount = input("Enter the discount percentage: ")
            if not new_discount.isdigit() or int(new_discount)==0:
                print("Please enter a valid number for the discount")
                continue
            else: break
        offer['discount'] = new_discount
    elif choice==2:
        while True:
            try:
                new_end_date = input("Enter the end date (YYYY-MM-DD): ")
                val_date= datetime.strptime(new_end_date, "%Y-%m-%d")
                break
            except ValueError:
                print("Please enter a valid date for the discount")
                continue

        offer['end_date'] = new_end_date
    write_csv(OFFERS_FILE, offers, fieldnames=offers[0].keys())
    print("Offer updated successfully!")

def main():
    while True:
        print("\nMain Menu")
        print("1. Register  2. Login  3. Admin Panel  4. Exit")
        choice = input("Enter choice: ")
        if choice == '1': register_user()
        elif choice == '2':
            user_id=login()
            if user_id: user_menu(user_id)
        elif choice == '3': admin_panel()
        elif choice == '4': break
        else: print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()


Rithika Baskaran
11:27 PM (3 minutes ago)
to Sujeeth




---------- Forwarded message ---------
From: Nangai J. <nangai.j@gmail.com>
Date: Sat, Aug 31, 2024 at 8:37 PM
Subject: updated project
To: Rithika Baskaran <rithi.basky@gmail.com>


import csv
import os
from datetime import datetime
from tabulate import tabulate
#to install tabulate , in command prompt , goto python dir, type   py -m pip install tabulate

# File paths
MENU_FILE = 'menu.csv'
TOPPINGS_FILE = 'toppings.csv'
SIDE_ORDERS_FILE = 'side_orders.csv'
OFFERS_FILE = 'offers.csv'
ORDERS_FILE = 'orders.csv'
ORDER_ITEMS_FILE = 'order_items.csv'
DELIVERED_ORDERS_FILE = 'delivered_orders.csv'
USERS_FILE = 'users.csv'
CART_FILE = 'cart.csv'

def read_csv(file_path):
    if not os.path.exists(file_path):
        return []
    with open(file_path, 'r', newline='') as file:
        reader = csv.DictReader(file)
        return list(reader)

def write_csv(file_path, data, fieldnames):
    with open(file_path, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

def get_next_id(file_path):
    data = read_csv(file_path)
    if not data: return 1
    return max(int(item['id']) for item in data) + 1

def register_user():
    users = read_csv(USERS_FILE)
    user_id = get_next_id(USERS_FILE)
    name=input("Enter your name: ")
    username = input("Enter username: ")
    password = input("Enter password: ")
    ph_no= input("Enter your phone number: ")
    address=input("Enter your address: ")
    email=input("Enter Email id: ")
    if any(not value for value in [name, username,password,ph_no,address,email]):
        print("Please try again")
        return
    if any(user['username'] == username for user in users):
        print("Username already exists.")
        return

    users.append({
        'id': user_id,
        'name': name,
        'username': username,
        'password': password,
        'phone': ph_no,
        'address': address,
        'email': email
    })
    write_csv(USERS_FILE, users, fieldnames=['id', 'name','username', 'password', 'phone','address', 'email'])
    print("User registered successfully!")

def login():
    users = read_csv(USERS_FILE)
    username = input("Enter username: ")
    password = input("Enter password: ")
    f=0
    for user in users:
        if user['username'] != username:
            continue
        else:
            f=1
            u=user['username']
            p=user['password']
            break
    if f==1:
        if p == password:
            print(f"Welcome {user['name']}!")
            return user['id']
        else:
            f=input("Forgot Password (yes/no): ")
            if f=='yes':
                new_password = input("Enter your new password: ")
                user['password'] = new_password
                write_csv(USERS_FILE, users, fieldnames=['id', 'name','username', 'password', 'phone', 'address','email'])
                return
            else:
                print("Pls try again")
                return
    else:
        f=input("Forgot Username (yes/no): ")
        if f=='yes':
            email = input("Enter your registered email address: ")
            if user['email'] == email:
                print(f"Your username is: {user['username']}")
                return
            print("Email address not found. Please try again.")
        else:
            print("Pls try again")
            return

def user_menu(user_id):
    if user_id:
        print("\nWelcome to Pizza Hub!!!")
        while True:
            print("1. View Pizza Hub Pamplet")
            print("2. Add to Cart  3. View Cart  4. Clear Cart  5. Place Order")
            print("6. View Ordered Details  7. Update Profile 8. Exit")
            customer_choice = input("Enter your choice: ")
            if customer_choice == '1':
                display_items(MENU_FILE, "Pizza Varieties")
                display_items(TOPPINGS_FILE, "Toppings")
                display_items(SIDE_ORDERS_FILE, "Snacks and Refreshments")
            elif customer_choice == '2': add_to_cart(user_id)
            elif customer_choice == '3': view_cart(user_id)
            elif customer_choice == '4': clear_cart(user_id)
            elif customer_choice == '5': place_order(user_id)
            elif customer_choice == '6': view_order_history(user_id)
            elif customer_choice == '7': update_profile(user_id)
            elif customer_choice == '8': break
            else: print("Invalid choice. Try again.")

# Function to update user profile information
def update_profile(user_id):
    users = read_csv(USERS_FILE)
    for user in users:
        if user['id'] == user_id:
            print("Your Profile:")
            print(f"Name: {user['name']}, Username: {user['username']}, Phone Number: {user['phone']}, Address: {user['address']}")
            print("\nUpdate Profile:")
            print("1. Phone Number  2. Address  3. Change Password  4. Email id  5. None")
            choice = input("Enter your choice: ")
            if choice == '1':
                phone_number = input("Enter new phone number: ")
                user['phone'] = phone_number
                print("Phone number updated successfully!")
            elif choice == '2':
                address = input("Enter new address: ")
                user['address'] = address
                print("Address updated successfully!")
            elif choice == '3':
                current_password = input("Enter current password: ")
                if current_password == user['password']:
                    new_password = input("Enter new password: ")
                    confirm_password = input("Confirm new password: ")
                    if new_password == confirm_password:
                        user['password'] = new_password
                        print("Password changed successfully!")
                    else: print("Passwords do not match.")
                else: print("Incorrect current password.")
            elif choice == '4':
                new_email = input("Enter new email id: ")
                user['email'] = new_email
                print("Email changed successfully!")
            elif choice==5: break
            else: print("Invalid choice.")
            break
    write_csv(USERS_FILE, users, fieldnames=['id', 'name','username', 'password', 'phone', 'address','email'])

def add_to_cart(user_id):
    menu = read_csv(MENU_FILE)
    toppings = read_csv(TOPPINGS_FILE)
    side_orders = read_csv(SIDE_ORDERS_FILE)
    cart = read_csv(CART_FILE)
    order_items = []
    total_price = 0
    while True:
        display_items(MENU_FILE, "Menu Items")
        item_id = input("Enter the menu item ID to order or type 'done' to finish: ")
        if item_id.lower() == 'done': break
        item = next((i for i in menu if i['id'] == item_id), None)
        quantity=0
        if not item:
            print("please give right input.")
            continue
        try:
            quantity = int(input(f"Enter quantity for {item['name']} pizza: "))
        except ValueError:
            print("quantity not entered. pls try again")
            continue
        if quantity==0:
            print("quantity not entered. pls try again")
            continue
        item_total_price = int(quantity) * float(item['price'])
        selected_toppings = []
        toppings_total_price = 0.0
        if item['category'].lower() == 'pizza':
            add_top = input("Do you want to add toppings? (yes/no): ").lower()
            if add_top not in ['yes','no']:
                print("please give right input.")
                continue
            if add_top == 'yes':
                while True:
                    print("\nAvailable Toppings:")
                    print(tabulate(toppings, headers="keys", tablefmt="grid"))
                    topping_id = input("Enter the topping ID to add or type 'done' to finish: ")
                    if topping_id.lower() == 'done' or topping_id == '': break
                    topping = next((t for t in toppings if t['id'] == topping_id), None)
                    if not topping:
                        print("Topping not found.")
                        continue
                    #topping_quantity = int(input(f"Enter quantity for {topping['name']}: "))
                    selected_toppings.append(topping['name'])
                    toppings_total_price += float(topping['price'])
            if not selected_toppings:
                selected_toppings.append('None')
            item_total_price += toppings_total_price * quantity
            total_price += item_total_price

            order_items.append({
            'menu_id': item_id,
            'category':'pizza',
            'quantity': quantity,
            'toppings': selected_toppings,
            'price': item_total_price
        })
    if not order_items:
        print("No Pizza selected.")

    side_orders = []
    while True:
        display_items(SIDE_ORDERS_FILE, "Side Orders")
        side_order_id = input("Enter the side order ID to order or type 'done' to finish: ")
        if side_order_id.lower() == 'done':
            break
        side_order = next((s for s in read_csv(SIDE_ORDERS_FILE) if s['id'] == side_order_id), None)
        if not side_order:
            print("Side order not found.")
            continue
        try:
            qty = int(input(f"Enter quantity for {side_order['name']}: "))
        except ValueError:
            print("quantity not entered. pls try again")
            continue
        if quantity==0:
            print("quantity not entered. pls try again")
            continue
        side_orders.append({
            'side_order_name': side_order['name'],
            'quantity': quantity
        })

        side_total_price =float(side_order['price'])* qty
        total_price +=side_total_price

        order_items.append({
            'menu_id': side_order_id,
            'category':'side_orders',
            'quantity': qty,
            'toppings': ['None'],
            'price': side_total_price
        })

    if not order_items:
        print("No items selected.")
        return

    if total_price > 0.0:
        for item in order_items:
            cart.append({
                'user_id': user_id,
                'menu_id': item['menu_id'],
                'category': item['category'],
                'quantity': item['quantity'],
                'toppings': ', '.join(item['toppings']),
                'price': item['price']
            })
        write_csv(CART_FILE, cart, fieldnames=['user_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
        print(f"Items added to cart successfully! Total price: ₹{total_price}")
    else: print("No items selected. Cart is not updated.")

def place_order(user_id):
    cart = read_csv(CART_FILE)
    offers = read_csv(OFFERS_FILE)
    user_cart = [item for item in cart if item['user_id'] == user_id]
    if not user_cart:
        print("Your cart is empty. Please add items to cart before placing an order.")
        return
    total_price = sum(float(item['price']) for item in user_cart if item['category'] == 'pizza' )

    display_items(OFFERS_FILE, "Offers")
    offer_id = input("Enter offer ID if you have one or type 'none': ")
    offer = next((o for o in offers if o['id'] == offer_id), None)
    discount=0.0
    if offer:
        offer_end_date = datetime.strptime(offer['end_date'], '%Y-%m-%d')
        if datetime.now() > offer_end_date:
            print("Offer expired!")
            return
        discount = float(offer['discount']) / 100
        total_price -= total_price * discount

        print("Price of pizza after discount: ",total_price)

    tot_price= sum(float(item['price']) for item in user_cart if item['category'] == 'side_orders' )
    total_price +=tot_price

    print("Total Price: ", total_price)

    order_id = get_next_id(ORDERS_FILE)
    orders = read_csv(ORDERS_FILE)
    orders.append({
        'id': order_id,
        'user_id': user_id,
        'total_price': total_price,
        'order_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'paid': 'yes',
    })

    write_csv(ORDERS_FILE, orders, fieldnames=['id', 'user_id', 'total_price', 'order_time', 'paid'])

    order_items_data = read_csv(ORDER_ITEMS_FILE)
    for item in user_cart:
        order_items_data.append({
            'order_id': order_id,
            'menu_id': item['menu_id'],
            'category':item['category'],
            'quantity': item['quantity'],
            'toppings': item['toppings'],
            'price': item['price']
        })
    write_csv(ORDER_ITEMS_FILE, order_items_data, fieldnames=['order_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
    generate_and_send_receipt(user_id, user_cart, total_price)

    cart = [item for item in cart if item['user_id'] != user_id]  # Clear user's cart
    write_csv(CART_FILE, cart, fieldnames=['user_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
    print("Order placed successfully!")

def generate_and_send_receipt(user_id, user_cart, total_price):
    users = read_csv(USERS_FILE)
    user = next((u for u in users if u['id'] == user_id), None)

    if user:
        receipt = f"Bill Receipt\n\nCustomer: {user['name']}\nPhone: {user['phone']}\n"
        receipt += f"Order Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        receipt += "\nItems:\n"

        menu = read_csv(MENU_FILE)
        side_orders = read_csv(SIDE_ORDERS_FILE)

        for item in user_cart:
            item_id = item['menu_id']
            category = item['category']
            if category == 'pizza':
                menu_item = next((i for i in menu if i['id'] == item_id), None)
                item_name = menu_item['name'] if menu_item else 'Unknown Pizza'
            else:
                side_item = next((i for i in side_orders if i['id'] == item_id), None)
                item_name = side_item['name'] if side_item else 'Unknown Side Order'

            receipt += f"{item_name} x {item['quantity']} - ₹{item['price']}\n"
            if item['toppings']:
                receipt += f"Toppings: {item['toppings']}\n"

        receipt += f"\nTotal Price: ₹{total_price}\n"

        print("\nSending Bill Receipt to Customer...\n")
        print(receipt)

# Function to view the order history
def view_order_history(user_id):
    orders = read_csv(ORDERS_FILE)
    order_items = read_csv(ORDER_ITEMS_FILE)
    menu = read_csv(MENU_FILE)
    side_orders = read_csv(SIDE_ORDERS_FILE)

    user_orders = [order for order in orders if order['user_id'] == user_id]
    if not user_orders:
        print("No order history found.")
        return
    print("\nYour Order History:")
    for order in user_orders:
        order_id = order['id']
        print(f"Order ID: {order_id}, Total Price: ₹{order['total_price']}, Order Time: {order['order_time']}")
        print("Items:")
        for item in order_items:
            if item['order_id'] == order_id:
                item_id = item['menu_id']
                category=item['category']
                m_id = next((i for i in menu if i['id'] == item_id and i['category']==category), None)
                if m_id !=None:
                    name=m_id['name']
                else:
                    m_id = next((i for i in side_orders if i['id'] == item_id), None)
                    name=m_id['name']
                print(f"Item Name: {name}, Quantity: {item['quantity']}, Toppings: {item['toppings']}, Price: ₹{item['price']}")

# Function to view the cart
def view_cart(user_id):
    cart = read_csv(CART_FILE)
    menu = read_csv(MENU_FILE)
    side_orders = read_csv(SIDE_ORDERS_FILE)
    user_cart = [item for item in cart if item['user_id'] == user_id]

    if not user_cart:
        print("Your cart is empty.")
        return
    total_price = sum(float(item['price']) for item in user_cart)

    print("\nYour Cart:")
    for item in user_cart:
        item_id = item['menu_id']
        category=item['category']
        m_id = next((i for i in menu if i['id'] == item_id and i['category']==category), None)
        if m_id !=None:
            name=m_id['name']
        else:
            m_id = next((i for i in side_orders if i['id'] == item_id), None)
            name=m_id['name']
        if item['category']=='pizza' and item['toppings'] !='None':
            print(f"Item Name: {name}, Quantity: {item['quantity']}, Toppings Added: {item['toppings']}, Price: ₹{item['price']}")
        else:
            print(f"Item Name: {name}, Quantity: {item['quantity']},  Price: ₹{item['price']}")
    print(f"Total Price: ₹{total_price}")

def clear_cart(user_id):
    cart = read_csv(CART_FILE)
    updated_cart = [item for item in cart if item['user_id'] != user_id]
    # Write the updated cart back to the file, removing the user's items
    write_csv(CART_FILE, updated_cart, fieldnames=['user_id', 'menu_id', 'quantity', 'price', 'toppings'])
    print("Cart cleared successfully!")

def admin_panel():
    passcode = input("Enter admin panel passcode: ")
    if passcode == 'a1b2c3':
        while True:
            print("\nAdmin Panel")
            print("1. View  2. Add  3.Remove  4.Update  5.Exit")
            choice = input("Enter choice: ")
            if choice == '1': view_options()
            elif choice == '2': add_options()
            elif choice == '3': remove_options()
            elif choice == '4': update_options()
            elif choice == '5': break
            else: print("Invalid choice. Try again.")
    else: print("Incorrect passcode. Access denied.")

def view_options():
    while True:
        print("\nView Options")
        print("1. Menu Items  2. Offers  3. Orders to Deliver  4. Delivered Orders  5. View Users  6. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1':
            display_items(MENU_FILE, "Menu Items")
            display_items(TOPPINGS_FILE, "Toppings")
            display_items(SIDE_ORDERS_FILE, "Side Orders")
        elif choice == '2': display_items(OFFERS_FILE, "Offers")
        elif choice == '3': display_items(ORDERS_FILE, "Orders to Deliver")
        elif choice == '4': display_items(DELIVERED_ORDERS_FILE, "Delivered Orders")
        elif choice == '5': display_user_details()
        elif choice  == '6': break
        else: print("Invalid choice. Try again.")

def add_options():
    while True:
        print("\nAdd Options")
        print("1. Menu Item  2. Topping  3. Side Order  4. Offer  5. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1': add_menu_item()
        elif choice == '2': add_topping()
        elif choice == '3': add_side_order()
        elif choice == '4': add_offer()
        elif choice == '5': break
        else: print("Invalid choice. Try again.")

def remove_options():
    while True:
        print("\nRemove Options")
        print("1. Menu Item  2. Topping  3. Side Order  4. Offer  5. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1': remove_item(MENU_FILE, "Menu Item")
        elif choice == '2': remove_item(TOPPINGS_FILE, "Topping")
        elif choice == '3': remove_item(SIDE_ORDERS_FILE, "Side Order")
        elif choice == '4': remove_item(OFFERS_FILE, "Offer")
        elif choice == '5': break
        else: print("Invalid choice. Try again.")

def update_options():
    while True:
        print("\nUpdate Options")
        print("1. Menu Item Price 2. Topping Price  3. Side Order Price  4. Offer Discount / End Date")
        print("5. Mark Orders as Delivered  6. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1': update_item(MENU_FILE, "Menu Item")
        elif choice == '2': update_item(TOPPINGS_FILE, "Topping")
        elif choice == '3': update_item(SIDE_ORDERS_FILE, "Side Order")
        elif choice == '4': update_offer()
        elif choice == '5': mark_order_as_delivered()
        elif choice == '6': break
        else: print("Invalid choice. Try again.")

def mark_order_as_delivered():
    display_items(ORDERS_FILE, "Orders to Deliver")
    while True:
        order_id = input("Enter the Order ID to mark as delivered or type 'done':: ")
        if order_id.lower() == 'done':
            break
        if order_id:
            orders = read_csv(ORDERS_FILE)
            order_items = read_csv(ORDER_ITEMS_FILE)
            delivered_orders = read_csv(DELIVERED_ORDERS_FILE)

            order = next((o for o in orders if o['id'] == order_id), None)
            if not order:
                print("Order ID not found.")
                continue
            d=get_next_id(DELIVERED_ORDERS_FILE)

            # Prepare the delivered order entry
            delivered_order = {
                'id': d,
                'order_id':order['id'],
                'user_id': order['user_id'],
                'total_amount': order['total_price'],
                'delivered_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            delivered_orders.append(delivered_order)
            write_csv(DELIVERED_ORDERS_FILE, delivered_orders, fieldnames=['id', 'order_id','user_id', 'total_amount', 'delivered_date'])

        # Remove the order from orders.csv
        orders = [o for o in orders if o['id'] != order_id]
        write_csv(ORDERS_FILE, orders, fieldnames=['id', 'user_id', 'total_price', 'order_time', 'paid'])
        # Remove the corresponding items from order_items.csv
        order_items = [item for item in order_items if item['order_id'] != order_id]
        write_csv(ORDER_ITEMS_FILE, order_items, fieldnames=['order_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
        print(f"Order {order_id} marked as delivered.")

def display_items(file_path, item_type):
    items = read_csv(file_path)
    if not items:
        print(f"No {item_type.lower()} available.")
        return
    print(f"\n{item_type}:")
    print(tabulate(items, headers="keys", tablefmt="grid"))

def display_user_details():
    users = read_csv(USERS_FILE)
    # Extract headers excluding the password
    headers = [key for key in users[0].keys() if key != 'password']
    # Extract data excluding the password
    user_data = [
        {key: user[key] for key in headers} for user in users
    ]
    # Use tabulate to display the user details in a table format
    if user_data:
        print(tabulate(user_data, headers="keys", tablefmt="grid"))
    else:
        print("No user details available.")



def add_menu_item():
    menu = read_csv(MENU_FILE)
    item_id = get_next_id(MENU_FILE)
    name = input("Enter the name of pizza: ")
    while True:
        price = input("Enter the price: ")
        if (not price.isdigit()) or float(price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break
    category = 'pizza'
    typ=input("Enter type of pizza(veg/non veg): ")
    size=input("Enter size(e.g. small/medium/large): ")
    if any(not value for value in [name, price, typ, size]):
        print("Please try again")
        return
    menu.append({
        'id': item_id,
        'name': name,
        'category': category,
        'type': typ,
        'size': size,
        'price': price

    })
    write_csv(MENU_FILE, menu, fieldnames=['id', 'name', 'category', 'type','size', 'price'])
    print("Menu item added successfully!")

def add_topping():
    toppings = read_csv(TOPPINGS_FILE)
    topping_id = get_next_id(TOPPINGS_FILE)
    name = input("Enter the topping name: ")
    while True:
        price = input("Enter the price: ")
        if not price.isdigit() or float(price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break
    if any(not value for value in [name, price]):
        print("Please try again")
        return

    toppings.append({
        'id': topping_id,
        'name': name,
        'price': price
    })
    write_csv(TOPPINGS_FILE, toppings, fieldnames=['id', 'name', 'price'])
    print("Topping added successfully!")

def add_side_order():
    side_orders = read_csv(SIDE_ORDERS_FILE)
    side_order_id = get_next_id(SIDE_ORDERS_FILE)
    name = input("Enter the side order name: ")
    while True:
        price = input("Enter the price: ")
        if not price.isdigit() or float(price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break
    size =input("Enter size(e.g. small/medium/4 count/6 pieces: ")
    category = input("Enter the category(drink/chicken wings/bread: ")
    if any(not value for value in [name, price, size, category]):
        print("Please try again")
        return

    side_orders.append({
        'id': side_order_id,
        'name': name,
        'category': category,
        'size': size,
        'price': price

    })
    write_csv(SIDE_ORDERS_FILE, side_orders, fieldnames=['id', 'name', 'category','size', 'price'])
    print("Side order added successfully!")

def add_offer():
    offers = read_csv(OFFERS_FILE)
    offer_id = get_next_id(OFFERS_FILE)
    offer_name= input("Enter offer name: ")
    while True:
        discount = input("Enter the discount percentage: ")
        if not discount.isdigit() or int(discount)==0:
            print("Please enter a valid number for the discount")
            continue
        else: break
    while True:
        try:
            end_date = input("Enter the end date (YYYY-MM-DD): ")
            val_date= datetime.strptime(end_date, "%Y-%m-%d")
            break
        except ValueError:
            print("Please enter a valid date for the discount")
            continue
    if any(not value for value in [offer_name, discount, end_date]):
        print("Please try again")
        return

    offers.append({
        'id': offer_id,
        'name': offer_name,
        'discount': discount,
        'end_date': end_date
    })
    write_csv(OFFERS_FILE, offers, fieldnames=['id','name', 'discount', 'end_date'])
    print("Offer added successfully!")

def remove_item(file_path, item_type):
    display_items(file_path, item_type)
    items = read_csv(file_path)
    item_id = input(f"Enter the {item_type.lower()} ID to remove: ")
    updated_items = [item for item in items if item['id'] != item_id]
    write_csv(file_path, updated_items, fieldnames=items[0].keys())
    print(f"{item_type} removed successfully!")

def update_item(file_path, item_type):
    display_items(file_path, item_type)
    items = read_csv(file_path)
    item_id = input(f"Enter the {item_type.lower()} ID to update price: ")
    item = next((i for i in items if i['id'] == item_id), None)
    if not item:
        print(f"{item_type} not found.")
        return
    while True:
        new_price = input(f"Enter new price for {item['name']}: ")
        if not new_price.isdigit() or float(new_price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break

    item['price'] = new_price
    write_csv(file_path, items, fieldnames=items[0].keys())
    print(f"{item_type} price updated successfully!")

def update_offer():
    offers = read_csv(OFFERS_FILE)
    offer_id = input("Enter the offer ID to update: ")
    offer = next((o for o in offers if o['id'] == offer_id), None)
    if not offer:
        print("Offer not found.")
        return
    print("1. Update Discount Percentage  2-New End date ")
    choice = input("Enter your choice: ")
    if choice==1:
        while True:
            new_discount = input("Enter the discount percentage: ")
            if not new_discount.isdigit() or int(new_discount)==0:
                print("Please enter a valid number for the discount")
                continue
            else: break
        offer['discount'] = new_discount
    elif choice==2:
        while True:
            try:
                new_end_date = input("Enter the end date (YYYY-MM-DD): ")
                val_date= datetime.strptime(new_end_date, "%Y-%m-%d")
                break
            except ValueError:
                print("Please enter a valid date for the discount")
                continue

        offer['end_date'] = new_end_date
    write_csv(OFFERS_FILE, offers, fieldnames=offers[0].keys())
    print("Offer updated successfully!")

def main():
    while True:
        print("\nMain Menu")
        print("1. Register  2. Login  3. Admin Panel  4. Exit")
        choice = input("Enter choice: ")
        if choice == '1': register_user()
        elif choice == '2':
            user_id=login()
            if user_id: user_menu(user_id)
        elif choice == '3': admin_panel()
        elif choice == '4': break
        else: print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()


SyntaxError: invalid decimal literal (<ipython-input-1-b39c61ba0296>, line 737)

In [ ]:




import csv
import os
from datetime import datetime
from tabulate import tabulate
#to install tabulate , in command prompt , goto python dir, type   py -m pip install tabulate

# File paths
MENU_FILE = 'menu.csv'
TOPPINGS_FILE = 'toppings.csv'
SIDE_ORDERS_FILE = 'side_orders.csv'
OFFERS_FILE = 'offers.csv'
ORDERS_FILE = 'orders.csv'
ORDER_ITEMS_FILE = 'order_items.csv'
DELIVERED_ORDERS_FILE = 'delivered_orders.csv'
USERS_FILE = 'users.csv'
CART_FILE = 'cart.csv'

def read_csv(file_path):
    if not os.path.exists(file_path):
        return []
    with open(file_path, 'r', newline='') as file:
        reader = csv.DictReader(file)
        return list(reader)

def write_csv(file_path, data, fieldnames):
    with open(file_path, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

def get_next_id(file_path):
    data = read_csv(file_path)
    if not data: return 1
    return max(int(item['id']) for item in data) + 1

def register_user():
    users = read_csv(USERS_FILE)
    user_id = get_next_id(USERS_FILE)
    name=input("Enter your name: ")
    username = input("Enter username: ")
    password = input("Enter password: ")
    ph_no= input("Enter your phone number: ")
    address=input("Enter your address: ")
    email=input("Enter Email id: ")
    if any(not value for value in [name, username,password,ph_no,address,email]):
        print("Please try again")
        return
    if any(user['username'] == username for user in users):
        print("Username already exists.")
        return

    users.append({
        'id': user_id,
        'name': name,
        'username': username,
        'password': password,
        'phone': ph_no,
        'address': address,
        'email': email
    })
    write_csv(USERS_FILE, users, fieldnames=['id', 'name','username', 'password', 'phone','address', 'email'])
    print("User registered successfully!")

def login():
    users = read_csv(USERS_FILE)
    username = input("Enter username: ")
    password = input("Enter password: ")
    f=0
    for user in users:
        if user['username'] != username:
            continue
        else:
            f=1
            u=user['username']
            p=user['password']
            break
    if f==1:
        if p == password:
            print(f"Welcome {user['name']}!")
            return user['id']
        else:
            f=input("Forgot Password (yes/no): ")
            if f=='yes':
                new_password = input("Enter your new password: ")
                user['password'] = new_password
                write_csv(USERS_FILE, users, fieldnames=['id', 'name','username', 'password', 'phone', 'address','email'])
                return
            else:
                print("Pls try again")
                return
    else:
        f=input("Forgot Username (yes/no): ")
        if f=='yes':
            email = input("Enter your registered email address: ")
            if user['email'] == email:
                print(f"Your username is: {user['username']}")
                return
            print("Email address not found. Please try again.")
        else:
            print("Pls try again")
            return

def user_menu(user_id):
    if user_id:
        print("\nWelcome to Pizza Hub!!!")
        while True:
            print("1. View Pizza Hub Pamplet")
            print("2. Add to Cart  3. View Cart  4. Clear Cart  5. Place Order")
            print("6. View Ordered Details  7. Update Profile 8. Exit")
            customer_choice = input("Enter your choice: ")
            if customer_choice == '1':
                display_items(MENU_FILE, "Pizza Varieties")
                display_items(TOPPINGS_FILE, "Toppings")
                display_items(SIDE_ORDERS_FILE, "Snacks and Refreshments")
            elif customer_choice == '2': add_to_cart(user_id)
            elif customer_choice == '3': view_cart(user_id)
            elif customer_choice == '4': clear_cart(user_id)
            elif customer_choice == '5': place_order(user_id)
            elif customer_choice == '6': view_order_history(user_id)
            elif customer_choice == '7': update_profile(user_id)
            elif customer_choice == '8': break
            else: print("Invalid choice. Try again.")

# Function to update user profile information
def update_profile(user_id):
    users = read_csv(USERS_FILE)
    for user in users:
        if user['id'] == user_id:
            print("Your Profile:")
            print(f"Name: {user['name']}, Username: {user['username']}, Phone Number: {user['phone']}, Address: {user['address']}")
            print("\nUpdate Profile:")
            print("1. Phone Number  2. Address  3. Change Password  4. Email id  5. None")
            choice = input("Enter your choice: ")
            if choice == '1':
                phone_number = input("Enter new phone number: ")
                user['phone'] = phone_number
                print("Phone number updated successfully!")
            elif choice == '2':
                address = input("Enter new address: ")
                user['address'] = address
                print("Address updated successfully!")
            elif choice == '3':
                current_password = input("Enter current password: ")
                if current_password == user['password']:
                    new_password = input("Enter new password: ")
                    confirm_password = input("Confirm new password: ")
                    if new_password == confirm_password:
                        user['password'] = new_password
                        print("Password changed successfully!")
                    else: print("Passwords do not match.")
                else: print("Incorrect current password.")
            elif choice == '4':
                new_email = input("Enter new email id: ")
                user['email'] = new_email
                print("Email changed successfully!")
            elif choice==5: break
            else: print("Invalid choice.")
            break
    write_csv(USERS_FILE, users, fieldnames=['id', 'name','username', 'password', 'phone', 'address','email'])

def add_to_cart(user_id):
    menu = read_csv(MENU_FILE)
    toppings = read_csv(TOPPINGS_FILE)
    side_orders = read_csv(SIDE_ORDERS_FILE)
    cart = read_csv(CART_FILE)
    order_items = []
    total_price = 0
    while True:
        display_items(MENU_FILE, "Menu Items")
        item_id = input("Enter the menu item ID to order or type 'done' to finish: ")
        if item_id.lower() == 'done': break
        item = next((i for i in menu if i['id'] == item_id), None)
        quantity=0
        if not item:
            print("please give right input.")
            continue
        try:
            quantity = int(input(f"Enter quantity for {item['name']} pizza: "))
        except ValueError:
            print("quantity not entered. pls try again")
            continue
        if quantity==0:
            print("quantity not entered. pls try again")
            continue
        item_total_price = int(quantity) * float(item['price'])
        selected_toppings = []
        toppings_total_price = 0.0
        if item['category'].lower() == 'pizza':
            add_top = input("Do you want to add toppings? (yes/no): ").lower()
            if add_top not in ['yes','no']:
                print("please give right input.")
                continue
            if add_top == 'yes':
                while True:
                    print("\nAvailable Toppings:")
                    print(tabulate(toppings, headers="keys", tablefmt="grid"))
                    topping_id = input("Enter the topping ID to add or type 'done' to finish: ")
                    if topping_id.lower() == 'done' or topping_id == '': break
                    topping = next((t for t in toppings if t['id'] == topping_id), None)
                    if not topping:
                        print("Topping not found.")
                        continue
                    #topping_quantity = int(input(f"Enter quantity for {topping['name']}: "))
                    selected_toppings.append(topping['name'])
                    toppings_total_price += float(topping['price'])
            if not selected_toppings:
                selected_toppings.append('None')
            item_total_price += toppings_total_price * quantity
            total_price += item_total_price

            order_items.append({
            'menu_id': item_id,
            'category':'pizza',
            'quantity': quantity,
            'toppings': selected_toppings,
            'price': item_total_price
        })
    if not order_items:
        print("No Pizza selected.")

    side_orders = []
    while True:
        display_items(SIDE_ORDERS_FILE, "Side Orders")
        side_order_id = input("Enter the side order ID to order or type 'done' to finish: ")
        if side_order_id.lower() == 'done':
            break
        side_order = next((s for s in read_csv(SIDE_ORDERS_FILE) if s['id'] == side_order_id), None)
        if not side_order:
            print("Side order not found.")
            continue
        try:
            qty = int(input(f"Enter quantity for {side_order['name']}: "))
        except ValueError:
            print("quantity not entered. pls try again")
            continue
        if quantity==0:
            print("quantity not entered. pls try again")
            continue
        side_orders.append({
            'side_order_name': side_order['name'],
            'quantity': quantity
        })

        side_total_price =float(side_order['price'])* qty
        total_price +=side_total_price

        order_items.append({
            'menu_id': side_order_id,
            'category':'side_orders',
            'quantity': qty,
            'toppings': ['None'],
            'price': side_total_price
        })

    if not order_items:
        print("No items selected.")
        return

    if total_price > 0.0:
        for item in order_items:
            cart.append({
                'user_id': user_id,
                'menu_id': item['menu_id'],
                'category': item['category'],
                'quantity': item['quantity'],
                'toppings': ', '.join(item['toppings']),
                'price': item['price']
            })
        write_csv(CART_FILE, cart, fieldnames=['user_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
        print(f"Items added to cart successfully! Total price: ₹{total_price}")
    else: print("No items selected. Cart is not updated.")

def place_order(user_id):
    cart = read_csv(CART_FILE)
    offers = read_csv(OFFERS_FILE)
    user_cart = [item for item in cart if item['user_id'] == user_id]
    if not user_cart:
        print("Your cart is empty. Please add items to cart before placing an order.")
        return
    total_price = sum(float(item['price']) for item in user_cart if item['category'] == 'pizza' )

    display_items(OFFERS_FILE, "Offers")
    offer_id = input("Enter offer ID if you have one or type 'none': ")
    offer = next((o for o in offers if o['id'] == offer_id), None)
    discount=0.0
    if offer:
        offer_end_date = datetime.strptime(offer['end_date'], '%Y-%m-%d')
        if datetime.now() > offer_end_date:
            print("Offer expired!")
            return
        discount = float(offer['discount']) / 100
        total_price -= total_price * discount

        print("Price of pizza after discount: ",total_price)

    tot_price= sum(float(item['price']) for item in user_cart if item['category'] == 'side_orders' )
    total_price +=tot_price

    print("Total Price: ", total_price)

    order_id = get_next_id(ORDERS_FILE)
    orders = read_csv(ORDERS_FILE)
    orders.append({
        'id': order_id,
        'user_id': user_id,
        'total_price': total_price,
        'order_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'paid': 'yes',
    })

    write_csv(ORDERS_FILE, orders, fieldnames=['id', 'user_id', 'total_price', 'order_time', 'paid'])

    order_items_data = read_csv(ORDER_ITEMS_FILE)
    for item in user_cart:
        order_items_data.append({
            'order_id': order_id,
            'menu_id': item['menu_id'],
            'category':item['category'],
            'quantity': item['quantity'],
            'toppings': item['toppings'],
            'price': item['price']
        })
    write_csv(ORDER_ITEMS_FILE, order_items_data, fieldnames=['order_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
    generate_and_send_receipt(user_id, user_cart, total_price)

    cart = [item for item in cart if item['user_id'] != user_id]  # Clear user's cart
    write_csv(CART_FILE, cart, fieldnames=['user_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
    print("Order placed successfully!")

def generate_and_send_receipt(user_id, user_cart, total_price):
    users = read_csv(USERS_FILE)
    user = next((u for u in users if u['id'] == user_id), None)

    if user:
        receipt = f"Bill Receipt\n\nCustomer: {user['name']}\nPhone: {user['phone']}\n"
        receipt += f"Order Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        receipt += "\nItems:\n"

        menu = read_csv(MENU_FILE)
        side_orders = read_csv(SIDE_ORDERS_FILE)

        for item in user_cart:
            item_id = item['menu_id']
            category = item['category']
            if category == 'pizza':
                menu_item = next((i for i in menu if i['id'] == item_id), None)
                item_name = menu_item['name'] if menu_item else 'Unknown Pizza'
            else:
                side_item = next((i for i in side_orders if i['id'] == item_id), None)
                item_name = side_item['name'] if side_item else 'Unknown Side Order'

            receipt += f"{item_name} x {item['quantity']} - ₹{item['price']}\n"
            if item['toppings']:
                receipt += f"Toppings: {item['toppings']}\n"

        receipt += f"\nTotal Price: ₹{total_price}\n"

        print("\nSending Bill Receipt to Customer...\n")
        print(receipt)

# Function to view the order history
def view_order_history(user_id):
    orders = read_csv(ORDERS_FILE)
    order_items = read_csv(ORDER_ITEMS_FILE)
    menu = read_csv(MENU_FILE)
    side_orders = read_csv(SIDE_ORDERS_FILE)

    user_orders = [order for order in orders if order['user_id'] == user_id]
    if not user_orders:
        print("No order history found.")
        return
    print("\nYour Order History:")
    for order in user_orders:
        order_id = order['id']
        print(f"Order ID: {order_id}, Total Price: ₹{order['total_price']}, Order Time: {order['order_time']}")
        print("Items:")
        for item in order_items:
            if item['order_id'] == order_id:
                item_id = item['menu_id']
                category=item['category']
                m_id = next((i for i in menu if i['id'] == item_id and i['category']==category), None)
                if m_id !=None:
                    name=m_id['name']
                else:
                    m_id = next((i for i in side_orders if i['id'] == item_id), None)
                    name=m_id['name']
                print(f"Item Name: {name}, Quantity: {item['quantity']}, Toppings: {item['toppings']}, Price: ₹{item['price']}")

# Function to view the cart
def view_cart(user_id):
    cart = read_csv(CART_FILE)
    menu = read_csv(MENU_FILE)
    side_orders = read_csv(SIDE_ORDERS_FILE)
    user_cart = [item for item in cart if item['user_id'] == user_id]

    if not user_cart:
        print("Your cart is empty.")
        return
    total_price = sum(float(item['price']) for item in user_cart)

    print("\nYour Cart:")
    for item in user_cart:
        item_id = item['menu_id']
        category=item['category']
        m_id = next((i for i in menu if i['id'] == item_id and i['category']==category), None)
        if m_id !=None:
            name=m_id['name']
        else:
            m_id = next((i for i in side_orders if i['id'] == item_id), None)
            name=m_id['name']
        if item['category']=='pizza' and item['toppings'] !='None':
            print(f"Item Name: {name}, Quantity: {item['quantity']}, Toppings Added: {item['toppings']}, Price: ₹{item['price']}")
        else:
            print(f"Item Name: {name}, Quantity: {item['quantity']},  Price: ₹{item['price']}")
    print(f"Total Price: ₹{total_price}")

def clear_cart(user_id):
    cart = read_csv(CART_FILE)
    updated_cart = [item for item in cart if item['user_id'] != user_id]
    # Write the updated cart back to the file, removing the user's items
    write_csv(CART_FILE, updated_cart, fieldnames=['user_id', 'menu_id', 'quantity', 'price', 'toppings'])
    print("Cart cleared successfully!")

def admin_panel():
    passcode = input("Enter admin panel passcode: ")
    if passcode == 'a1b2c3':
        while True:
            print("\nAdmin Panel")
            print("1. View  2. Add  3.Remove  4.Update  5.Exit")
            choice = input("Enter choice: ")
            if choice == '1': view_options()
            elif choice == '2': add_options()
            elif choice == '3': remove_options()
            elif choice == '4': update_options()
            elif choice == '5': break
            else: print("Invalid choice. Try again.")
    else: print("Incorrect passcode. Access denied.")

def view_options():
    while True:
        print("\nView Options")
        print("1. Menu Items  2. Offers  3. Orders to Deliver  4. Delivered Orders  5. View Users  6. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1':
            display_items(MENU_FILE, "Menu Items")
            display_items(TOPPINGS_FILE, "Toppings")
            display_items(SIDE_ORDERS_FILE, "Side Orders")
        elif choice == '2': display_items(OFFERS_FILE, "Offers")
        elif choice == '3': display_items(ORDERS_FILE, "Orders to Deliver")
        elif choice == '4': display_items(DELIVERED_ORDERS_FILE, "Delivered Orders")
        elif choice == '5': display_user_details()
        elif choice  == '6': break
        else: print("Invalid choice. Try again.")

def add_options():
    while True:
        print("\nAdd Options")
        print("1. Menu Item  2. Topping  3. Side Order  4. Offer  5. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1': add_menu_item()
        elif choice == '2': add_topping()
        elif choice == '3': add_side_order()
        elif choice == '4': add_offer()
        elif choice == '5': break
        else: print("Invalid choice. Try again.")

def remove_options():
    while True:
        print("\nRemove Options")
        print("1. Menu Item  2. Topping  3. Side Order  4. Offer  5. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1': remove_item(MENU_FILE, "Menu Item")
        elif choice == '2': remove_item(TOPPINGS_FILE, "Topping")
        elif choice == '3': remove_item(SIDE_ORDERS_FILE, "Side Order")
        elif choice == '4': remove_item(OFFERS_FILE, "Offer")
        elif choice == '5': break
        else: print("Invalid choice. Try again.")

def update_options():
    while True:
        print("\nUpdate Options")
        print("1. Menu Item Price 2. Topping Price  3. Side Order Price  4. Offer Discount / End Date")
        print("5. Mark Orders as Delivered  6. Back to Admin Panel")
        choice = input("Enter choice: ")
        if choice == '1': update_item(MENU_FILE, "Menu Item")
        elif choice == '2': update_item(TOPPINGS_FILE, "Topping")
        elif choice == '3': update_item(SIDE_ORDERS_FILE, "Side Order")
        elif choice == '4': update_offer()
        elif choice == '5': mark_order_as_delivered()
        elif choice == '6': break
        else: print("Invalid choice. Try again.")

def mark_order_as_delivered():
    display_items(ORDERS_FILE, "Orders to Deliver")
    while True:
        order_id = input("Enter the Order ID to mark as delivered or type 'done':: ")
        if order_id.lower() == 'done':
            break
        if order_id:
            orders = read_csv(ORDERS_FILE)
            order_items = read_csv(ORDER_ITEMS_FILE)
            delivered_orders = read_csv(DELIVERED_ORDERS_FILE)

            order = next((o for o in orders if o['id'] == order_id), None)
            if not order:
                print("Order ID not found.")
                continue
            d=get_next_id(DELIVERED_ORDERS_FILE)

            # Prepare the delivered order entry
            delivered_order = {
                'id': d,
                'order_id':order['id'],
                'user_id': order['user_id'],
                'total_amount': order['total_price'],
                'delivered_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            delivered_orders.append(delivered_order)
            write_csv(DELIVERED_ORDERS_FILE, delivered_orders, fieldnames=['id', 'order_id','user_id', 'total_amount', 'delivered_date'])

        # Remove the order from orders.csv
        orders = [o for o in orders if o['id'] != order_id]
        write_csv(ORDERS_FILE, orders, fieldnames=['id', 'user_id', 'total_price', 'order_time', 'paid'])
        # Remove the corresponding items from order_items.csv
        order_items = [item for item in order_items if item['order_id'] != order_id]
        write_csv(ORDER_ITEMS_FILE, order_items, fieldnames=['order_id', 'menu_id', 'category','quantity', 'toppings', 'price'])
        print(f"Order {order_id} marked as delivered.")

def display_items(file_path, item_type):
    items = read_csv(file_path)
    if not items:
        print(f"No {item_type.lower()} available.")
        return
    print(f"\n{item_type}:")
    print(tabulate(items, headers="keys", tablefmt="grid"))

def display_user_details():
    users = read_csv(USERS_FILE)
    # Extract headers excluding the password
    headers = [key for key in users[0].keys() if key != 'password']
    # Extract data excluding the password
    user_data = [
        {key: user[key] for key in headers} for user in users
    ]
    # Use tabulate to display the user details in a table format
    if user_data:
        print(tabulate(user_data, headers="keys", tablefmt="grid"))
    else:
        print("No user details available.")



def add_menu_item():
    menu = read_csv(MENU_FILE)
    item_id = get_next_id(MENU_FILE)
    name = input("Enter the name of pizza: ")
    while True:
        price = input("Enter the price: ")
        if (not price.isdigit()) or float(price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break
    category = 'pizza'
    typ=input("Enter type of pizza(veg/non veg): ")
    size=input("Enter size(e.g. small/medium/large): ")
    if any(not value for value in [name, price, typ, size]):
        print("Please try again")
        return
    menu.append({
        'id': item_id,
        'name': name,
        'category': category,
        'type': typ,
        'size': size,
        'price': price

    })
    write_csv(MENU_FILE, menu, fieldnames=['id', 'name', 'category', 'type','size', 'price'])
    print("Menu item added successfully!")

def add_topping():
    toppings = read_csv(TOPPINGS_FILE)
    topping_id = get_next_id(TOPPINGS_FILE)
    name = input("Enter the topping name: ")
    while True:
        price = input("Enter the price: ")
        if not price.isdigit() or float(price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break
    if any(not value for value in [name, price]):
        print("Please try again")
        return

    toppings.append({
        'id': topping_id,
        'name': name,
        'price': price
    })
    write_csv(TOPPINGS_FILE, toppings, fieldnames=['id', 'name', 'price'])
    print("Topping added successfully!")

def add_side_order():
    side_orders = read_csv(SIDE_ORDERS_FILE)
    side_order_id = get_next_id(SIDE_ORDERS_FILE)
    name = input("Enter the side order name: ")
    while True:
        price = input("Enter the price: ")
        if not price.isdigit() or float(price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break
    size =input("Enter size(e.g. small/medium/4 count/6 pieces: ")
    category = input("Enter the category(drink/chicken wings/bread: ")
    if any(not value for value in [name, price, size, category]):
        print("Please try again")
        return

    side_orders.append({
        'id': side_order_id,
        'name': name,
        'category': category,
        'size': size,
        'price': price

    })
    write_csv(SIDE_ORDERS_FILE, side_orders, fieldnames=['id', 'name', 'category','size', 'price'])
    print("Side order added successfully!")

def add_offer():
    offers = read_csv(OFFERS_FILE)
    offer_id = get_next_id(OFFERS_FILE)
    offer_name= input("Enter offer name: ")
    while True:
        discount = input("Enter the discount percentage: ")
        if not discount.isdigit() or int(discount)==0:
            print("Please enter a valid number for the discount")
            continue
        else: break
    while True:
        try:
            end_date = input("Enter the end date (YYYY-MM-DD): ")
            val_date= datetime.strptime(end_date, "%Y-%m-%d")
            break
        except ValueError:
            print("Please enter a valid date for the discount")
            continue
    if any(not value for value in [offer_name, discount, end_date]):
        print("Please try again")
        return

    offers.append({
        'id': offer_id,
        'name': offer_name,
        'discount': discount,
        'end_date': end_date
    })
    write_csv(OFFERS_FILE, offers, fieldnames=['id','name', 'discount', 'end_date'])
    print("Offer added successfully!")

def remove_item(file_path, item_type):
    display_items(file_path, item_type)
    items = read_csv(file_path)
    item_id = input(f"Enter the {item_type.lower()} ID to remove: ")
    updated_items = [item for item in items if item['id'] != item_id]
    write_csv(file_path, updated_items, fieldnames=items[0].keys())
    print(f"{item_type} removed successfully!")

def update_item(file_path, item_type):
    display_items(file_path, item_type)
    items = read_csv(file_path)
    item_id = input(f"Enter the {item_type.lower()} ID to update price: ")
    item = next((i for i in items if i['id'] == item_id), None)
    if not item:
        print(f"{item_type} not found.")
        return
    while True:
        new_price = input(f"Enter new price for {item['name']}: ")
        if not new_price.isdigit() or float(new_price)==0:
            print("Please enter a valid number for the price")
            continue
        else: break

    item['price'] = new_price
    write_csv(file_path, items, fieldnames=items[0].keys())
    print(f"{item_type} price updated successfully!")

def update_offer():
    offers = read_csv(OFFERS_FILE)
    offer_id = input("Enter the offer ID to update: ")
    offer = next((o for o in offers if o['id'] == offer_id), None)
    if not offer:
        print("Offer not found.")
        return
    print("1. Update Discount Percentage  2-New End date ")
    choice = input("Enter your choice: ")
    if choice==1:
        while True:
            new_discount = input("Enter the discount percentage: ")
            if not new_discount.isdigit() or int(new_discount)==0:
                print("Please enter a valid number for the discount")
                continue
            else: break
        offer['discount'] = new_discount
    elif choice==2:
        while True:
            try:
                new_end_date = input("Enter the end date (YYYY-MM-DD): ")
                val_date= datetime.strptime(new_end_date, "%Y-%m-%d")
                break
            except ValueError:
                print("Please enter a valid date for the discount")
                continue

        offer['end_date'] = new_end_date
    write_csv(OFFERS_FILE, offers, fieldnames=offers[0].keys())
    print("Offer updated successfully!")

def main():
    while True:
        print("\nMain Menu")
        print("1. Register  2. Login  3. Admin Panel  4. Exit")
        choice = input("Enter choice: ")
        if choice == '1': register_user()
        elif choice == '2':
            user_id=login()
            if user_id: user_menu(user_id)
        elif choice == '3': admin_panel()
        elif choice == '4': break
        else: print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()



Main Menu
1. Register  2. Login  3. Admin Panel  4. Exit
